## Setup

In [1]:
!pip install -q -U -r requirements.txt
!pip install wandb


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
from pathlib import Path
from getpass import getpass
from datasets import load_dataset

import pandas as pd
import shutil
from tqdm.auto import tqdm
import ast
import pybboxes as pbx

import ultralytics
from ultralytics import YOLO

import wandb

import random

os.environ["WANDB_PROJECT"] = "object-detection-yolov8"
os.environ["WANDB_NOTEBOOK_NAME"] = "yolov8_bloodcells_finetuning.ipynb"

In [4]:
# run = wandb.init()
# run.finish()

In [62]:
from ultralytics.yolo.utils.torch_utils import get_flops, get_num_params

# try:
#     import wandb

#     assert hasattr(wandb, '__version__')
# except (ImportError, AssertionError):
#     wandb = None


def on_pretrain_routine_start(trainer):
    wandb.init(
        project=os.getenv("WANDB_PROJECT"), 
        # project=trainer.args.project or "YOLOv8", 
        name=trainer.args.name, 
        save_code=False,
        config=dict(trainer.args)
    )
    # ) if not wandb.run else wandb.run


def on_fit_epoch_end(trainer):
    wandb.run.log(trainer.metrics, step=trainer.epoch + 1)

    if trainer.epoch == 0:
        model_info = {
            "model/parameters": get_num_params(trainer.model),
            "model/GFLOPs": round(get_flops(trainer.model), 3),
            "model/speed(ms)": round(sum(trainer.validator.speed.values()), 3)
        }
        wandb.run.log(model_info, step=trainer.epoch + 1)

 
def on_train_epoch_end(trainer):
    wandb.run.log(trainer.label_loss_items(trainer.tloss, prefix="train"), step=trainer.epoch + 1)
    wandb.run.log(trainer.lr, step=trainer.epoch + 1)

    if trainer.epoch == 0:
        print(f"[INFO] Logging atraining artifacts for Epoch {trainer.epoch + 1}...")
        # wandb.run.log({f.stem: wandb.Image(str(f)) for f in trainer.save_dir.glob('train_batch*.jpg')}, step=trainer.epoch + 1)
        images = [wandb.Image(str(img_path), caption=img_path.stem) for img_path in Path(trainer.save_dir).glob("*.jpg")]
        wandb.run.log({f"training artifacts": images}, step=trainer.epoch + 1)


def on_train_end(trainer):
    art = wandb.Artifact(type="model", name=f"run_{wandb.run.id}_model")
    if trainer.best.exists():
        art.add_file(trainer.best)
        wandb.run.log_artifact(art)
    


def teardown(trainer):
    wandb.finish()

def on_predict_start(predictor):
    wandb.init(
        project=os.getenv("WANDB_PROJECT"), 
        # name=predictor.args.name,
        job_type="prediction",
        tags=["prediction", predictor.args.name],
        save_code=True,
        config=dict(predictor.args)
    )
    # ) if not wandb.run else wandb.run

def on_predict_end(predictor):
    split = Path(predictor.data_path).parent.stem
    images = [wandb.Image(str(img_path), caption=img_path.stem) for img_path in Path(predictor.save_dir).glob("*.jpg")]
    wandb.run.log({f"{split} set | {predictor.args.name} | pred": images})
    wandb.finish()


callbacks = {
    "on_pretrain_routine_start": on_pretrain_routine_start,
    "teardown": teardown,
    "on_train_epoch_end": on_train_epoch_end,
    "on_fit_epoch_end": on_fit_epoch_end,
    "on_train_end": on_train_end,
    "on_predict_start": on_predict_start,
    "on_predict_end": on_predict_end,
} if wandb else {}

In [36]:
# callbacks = {
#     "on_pretrain_routine_start": on_pretrain_routine_start,
#     "on_train_epoch_end": on_train_epoch_end,
#     "on_fit_epoch_end": on_fit_epoch_end,
#     "on_train_end": on_train_end,
#     "on_predict_start": on_predict_start,
#     "on_predict_end": on_predict_end,
# } if wandb else {}

In [8]:
dataset = load_dataset("keremberke/blood-cell-object-detection", "full")

In [13]:
! rm -rf workspace/object-detection-balloons/datasets

In [14]:
dataset_dir = "/workspace/object-detection-balloons/datasets/bloodcells"
os.makedirs(dataset_dir, exist_ok=True)

In [15]:
# rm -rf /workspace/object-detection-balloons/datasets/bloodcells

In [16]:
for split in dataset:
    for row in dataset[split]:
        img_path = Path(os.path.join(dataset_dir, "images", split, str(row["image_id"])) + ".jpg")
        os.makedirs(img_path.parent, exist_ok=True)
        row["image"].save(img_path)
        for bbox, category in zip(row["objects"]["bbox"], row["objects"]["category"]):
            bbox_yolo = pbx.convert_bbox(bbox, from_type="coco", to_type="yolo", image_size=(row["width"], row["height"]))
            bbox_yolo = " ".join([str(bb) for bb in bbox_yolo])
            file_path = Path(os.path.join(dataset_dir, "labels", split, str(row["image_id"])) + ".txt")
            os.makedirs(file_path.parent, exist_ok=True)
            with open(file_path, "a") as f:
                f.write(f"{category} {bbox_yolo}\n")

In [55]:
ultralytics.checks()

Ultralytics YOLOv8.0.173 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
Setup complete ✅ (12 CPUs, 62.8 GB RAM, 0.9/20.0 GB disk)


In [60]:
train_img_base_path = "/workspace/object-detection-balloons/datasets/bloodcells/images/train"
train_img_paths = [os.path.join(train_img_base_path, fname) for fname in os.listdir(train_img_base_path)]
print(train_img_paths[:3])

val_img_base_path = "/workspace/object-detection-balloons/datasets/bloodcells/images/validation"
val_img_paths = [os.path.join(val_img_base_path, fname) for fname in os.listdir(val_img_base_path)]
print(val_img_paths[:3])

test_img_base_path = "/workspace/object-detection-balloons/datasets/bloodcells/images/test"
test_img_paths = [os.path.join(test_img_base_path, fname) for fname in os.listdir(test_img_base_path)]
print(test_img_paths[:3])

['/workspace/object-detection-balloons/datasets/bloodcells/images/train/3.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/train/129.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/train/153.jpg']
['/workspace/object-detection-balloons/datasets/bloodcells/images/validation/51.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/validation/14.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/validation/48.jpg']
['/workspace/object-detection-balloons/datasets/bloodcells/images/test/10.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/test/32.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/test/34.jpg']


In [19]:
# for split in dataset:
#     for row in dataset[split]:
#         img_path = Path(os.path.join(dataset_dir, "images", split, str(row["image_id"])) + ".jpg")
#         os.makedirs(img_path.parent, exist_ok=True)
#         row["image"].save(img_path)
#         for bbox, category in zip(row["objects"]["bbox"], row["objects"]["category"]):
#             bbox_yolo = pbx.convert_bbox(bbox, from_type="coco", to_type="yolo", image_size=(row["width"], row["height"]))
#             bbox_yolo = " ".join([str(bb) for bb in bbox_yolo])
#             file_path = Path(os.path.join(dataset_dir, "labels", split, str(row["image_id"])) + ".txt")
#             os.makedirs(file_path.parent, exist_ok=True)
#             with open(file_path, "a") as f:
#                 f.write(f"{category} {bbox_yolo}\n")

## Log images on WANDB

In [ ]:
run = wandb.init(job_type="dataset")

class_id_to_label = {
    0: "platelets",
    1: "rbc",
    2: "wbc",
}

images = []
for split in tqdm(dataset):
    for row in dataset[split]:
        positions = [
            dict(zip(("minX", "minY", "maxX", "maxY"), pbx.convert_bbox(bbox, from_type="coco", to_type="voc", image_size=(row["width"], row["height"]))))
            for bbox in row["objects"]["bbox"]
        ]

        class_ids = row["objects"]["category"]
        box_captions = [class_id_to_label[id] for id in class_ids]

        box_data = [dict(zip(("position", "class_id", "box_caption", "domain"), x)) for x in zip(positions, class_ids, box_captions, ["pixel"] * len(class_ids))]

        images.append(wandb.Image(row["image"], caption=str(row["image_id"]), boxes={"ground_truth": {"box_data": box_data}}))

    # run.finish()
    # break
    run.log({f"{split}_set": images})
    images = []

run.finish()

## Baseline

In [8]:
checkpoint = 'yolov8n.pt'

In [31]:
yolo_base = YOLO(checkpoint)

for cb_name, fb_fn in callbacks.items():
    yolo_base.add_callback(cb_name, fb_fn)

preds = yolo_base.predict(val_img_paths, save=True, project="preds", name="baseline")


0: 640x640 1 toothbrush, 1: 640x640 (no detections), 2: 640x640 1 person, 3: 640x640 1 teddy bear, 4: 640x640 1 person, 5: 640x640 (no detections), 6: 640x640 2 persons, 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 1 person, 11: 640x640 1 person, 12: 640x640 2 persons, 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 1 person, 16: 640x640 (no detections), 17: 640x640 (no detections), 18: 640x640 1 person, 1 sports ball, 19: 640x640 1 person, 20: 640x640 1 toothbrush, 21: 640x640 1 person, 22: 640x640 (no detections), 23: 640x640 1 toothbrush, 24: 640x640 1 person, 1 toothbrush, 25: 640x640 1 donut, 26: 640x640 2 persons, 27: 640x640 (no detections), 28: 640x640 (no detections), 29: 640x640 (no detections), 30: 640x640 2 sports balls, 1 toothbrush, 31: 640x640 1 donut, 32: 640x640 1 sports ball, 33: 640x640 1 person, 1 sports ball, 34: 640x640 (no detections), 35: 640x640 (no detections), 36: 640x640 (no detections

## Finetuning

In [63]:
yolo_finetuned = YOLO(checkpoint)  # load a pretrained model (recommended for training)
for cb_name, cb_fn in callbacks.items():
    yolo_finetuned.add_callback(cb_name, cb_fn)

dataset_yaml_path = "/workspace/object-detection-balloons/bloodcells.yaml"

In [64]:
# Use the model
results = yolo_finetuned.train(data=dataset_yaml_path, epochs=10, batch=16)  # train the model

Ultralytics YOLOv8.0.173 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/workspace/object-detection-balloons/bloodcells.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, forma

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /workspace/object-detection-balloons/datasets/bloodcells/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
val: Scanning /workspace/object-detection-balloons/datasets/bloodcells/labels/validation.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
Plotting labels to runs/detect/train22/labels.jpg... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train22
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/10      2.25G      1.557      3.162      1.632        210        640: 100%|██████████| 

[INFO] Logging atraining artifacts for Epoch 1...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.98it/s]
                   all         73        967     0.0433      0.633      0.164     0.0826

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/10       2.3G      1.274      1.666      1.378        193        640: 100%|██████████| 16/16 [00:01<00:00,  8.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  5.11it/s]
                   all         73        967      0.884      0.334      0.571      0.385

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/10      2.26G      1.185      1.295      1.323        162        640: 100%|██████████| 16/16 [00:01<00:00,  9.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.95it/s]
  

AttributeError: 'DetectionTrainer' object has no attribute 'predict'

## Evaluation

### Validation Eval

In [ ]:
!rm -rf /workspace/object-detection-balloons/preds/val
preds = yolo_finetuned.predict(val_img_paths, save=True, project="preds", name="val") 

### Test Eval

In [ ]:
!rm -rf /workspace/object-detection-balloons/preds/test
preds = yolo_finetuned.predict(test_img_paths, save=True, project="preds", name="test") 